In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import cv2
import os
from PIL import Image, ImageDraw
%matplotlib inline

In [17]:
drive_id = 'HT096_1505983158'
lane_type= ['solid','dash']
color_type = ['red','green']


In [18]:
import shutil
save_dir = 'visualization/{}/'.format(drive_id)
try:
    shutil.rmtree(save_dir)
except:
    pass
# if not os.path.exists(save_dir):
os.makedirs(save_dir)
os.makedirs(save_dir+'topdown/')
for lane in lane_type:
    os.makedirs(save_dir+'prediction_{}/'.format(lane))

# copy paste the rgb topdown images to the result directory
for filename in glob.glob('{}/topdown-img/RGB/*.png'.format(drive_id)):
    shutil.copy(filename, save_dir+'topdown/')

for lane, color in zip(lane_type,color_type):
    file_list=glob.glob('{}/connection/{}/*'.format(drive_id,lane))    
    for counter,file_list_elem in enumerate(file_list):
        lines = glob.glob(file_list_elem+'/*.rowcol')
        time_stamp = file_list_elem.split('/')[-1]

        if len(lines)==0:
            continue
        # td_img = cv2.imread('{}/topdown-img/RGB/{}.png'.format(drive_id,time_stamp))
        try:
            im_td = Image.open(save_dir+'/topdown/{}.png'.format(time_stamp))
            im_td=im_td.resize((1024,1024))
            draw_td = ImageDraw.Draw(im_td)
        except:
            continue
        for line in lines:
            single_line = pd.read_csv(line,skiprows=0,delim_whitespace=True)
            single_line_m=single_line.values

            for index in range(1,len(single_line)):
                draw_td.line(tuple(single_line_m[index-1][::-1]) + tuple(single_line_m[index][::-1]), fill=color, width=1)
        im_td.save(save_dir+'topdown/{}.png'.format(time_stamp))

    #     for lane in lane_type:
        im_prd = Image.open('{}/prediction/results/{}/{}.png'.format(drive_id,lane,time_stamp)).convert('RGB')
        draw_prd = ImageDraw.Draw(im_prd)

        for line in lines:
            single_line = pd.read_csv(line,skiprows=0,delim_whitespace=True)
            single_line_m=single_line.values

            for index in range(1,len(single_line)):
                draw_prd.line(tuple(single_line_m[index-1][::-1]) + tuple(single_line_m[index][::-1]), fill="red", width=1)

        im_prd.save(save_dir+'prediction_{}/{}.png'.format(lane,time_stamp))